Caricate il data set Arrythmia in una DataFrame Pandas.

In [15]:
import pandas as pd
import numpy as np

labels=list(range(279)) + ['class']

df = pd.read_csv('/content/drive/MyDrive/data/_arrhythmia/arrhythmia.csv', names=labels)

Effettuate la pulizia dei dati eliminando le colonne che contengono più di un '?'. Nelle colonne che contengono un solo '?' sostituitelo col valor medio della colonna.

In [16]:
df

,0,1,2,3,4,5,6,7,8,9,...,270,271,272,273,274,275,276,277,278,class
0,75,0,190,80,91,193,371,174,121,-16,...,0.0,9.0,-0.9,0.0,0.0,0.9,2.9,23.3,49.4,8
1,56,1,165,64,81,174,401,149,39,25,...,0.0,8.5,0.0,0.0,0.0,0.2,2.1,20.4,38.8,6
2,54,0,172,95,138,163,386,185,102,96,...,0.0,9.5,-2.4,0.0,0.0,0.3,3.4,12.3,49.0,10
3,55,0,175,94,100,202,380,179,143,28,...,0.0,12.2,-2.2,0.0,0.0,0.4,2.6,34.6,61.6,1
4,75,0,190,80,88,181,360,177,103,-16,...,0.0,13.1,-3.6,0.0,0.0,-0.1,3.9,25.4,62.8,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,53,1,160,70,80,199,382,154,117,-37,...,0.0,4.3,-5.0,0.0,0.0,0.7,0.6,-4.4,-0.5,1
448,37,0,190,85,100,137,361,201,73,86,...,0.0,15.6,-1.6,0.0,0.0,0.4,2.4,38.0,62.4,10
449,36,0,166,68,108,176,365,194,116,-85,...,0.0,16.3,-28.6,0.0,0.0,1.5,1.0,-44.2,-33.2,2
450,32,1,155,55,93,106,386,218,63,54,...,-0.4,12.0,-0.7,0.0,0.0,0.5,2.4,25.0,46.6,1


In [17]:
df2 = df.copy()
to_drop = []
# Elimino colonne con più di un '?'
for col in range(df.shape[1]-1):
  not_valid = df[df.iloc[:, col]=='?'].shape[0]
  if not_valid>1:
    print("dropping col", col, not_valid)
    to_drop.append(col)

df2 = df2.drop(columns=to_drop)
print(df2.shape)

# Sostituisco in df2 '?' con media colonna
m = df2.mean(numeric_only=True)
for col in range(df2.shape[1]-1):
  for i in range(df2.shape[0]):
    if df2.iat[i, col] == '?':
      print("sostituito", i, col, "con", m.iloc[col])
      df2.iat[i, col] = m.iloc[col]

df2 = df2.astype(float)

dropping col 10 8
dropping col 11 22
dropping col 13 376
(452, 277)
sostituito 241 10 con 5.628318584070796
sostituito 4 11 con 51.6283185840708


Sviluppate classificatori k-NN per classificare i dati con uno split train/test 60/40 per diversi valori di k e determinate il migliore (in base all'accuracy).

In [18]:
# Estraggo i dati dal DataFrame
target = df2.iloc[:, -1].to_numpy(copy=True)
data = df2.iloc[:, 0:-1].to_numpy(copy=True)
print(target.shape, data.shape)

# Faccio lo split fra train e test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=11, train_size=0.6)

# Valuto diversi modelli al variare di k
from sklearn.neighbors import KNeighborsClassifier
for k in range(1, 10):
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(X=X_train, y=y_train)
  # Stampo le performance
  print(f'{k}: {knn.score(X_test, y_test):.2%}')

(452,) (452, 276)
1: 55.80%
2: 57.46%
3: 62.43%
4: 58.01%
5: 56.91%
6: 56.35%
7: 54.70%
8: 55.25%
9: 54.14%


Effettuate una riduzione di dimensionalità con PCA in modo da mantenere il 90% della varianza ed effettuate nuovamente la classificazione k-NN per il solo valore migliore di k trovato in precedenza.

In [19]:
# Trasformo i dati con PCA mantenendo 90% della varianza
from sklearn.decomposition import PCA
pca = PCA(n_components=0.90, svd_solver='full')
data_n = pca.fit(data).transform(data)
print(pca.n_components_)

# Faccio lo split fra train e test
from sklearn.model_selection import train_test_split
Xn_train, Xn_test, yn_train, yn_test = train_test_split(data_n, target, random_state=11, train_size=0.6)

# Costruisco un modello
from sklearn.neighbors import KNeighborsClassifier
knn2 = KNeighborsClassifier(n_neighbors=3)
knn2.fit(X=Xn_train, y=yn_train)

# Stampo le performance
print(f'{knn2.score(Xn_test, yn_test):.2%}')

27
64.09%


Riportate infine la matrice di confusione dei classificatori col miglior valore di k sia nel caso a dimensionalità piena che a dimensionalità ridotta.

In [22]:
from sklearn.metrics import confusion_matrix

# Calcolo la matrice di confusione del classificatore a dimensionalità piena per k=5
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X=X_train, y=y_train)
predicted = knn.predict(X=X_test)
expected = y_test
confusion = confusion_matrix(y_true=expected, y_pred=predicted)
confusion

array([[91,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0],
       [17,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 3,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 3,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0],
       [11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 2,  0,  0,  0,  0,  0,  0,  4,  0,  0,  0,  0],
       [11,  2,  0,  0,  0,  0,  0,  0,  6,  0,  0,  0],
       [ 3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0],
       [ 7,  0,  0,  0,  0,  0,  0,  0,  1,  0,  1,  0]])

In [23]:
# Calcolo la matrice di confusione del classificatore a dimensionalità ridotta per k=3
predicted = knn2.predict(X=Xn_test)
expected = yn_test
confusion2 = confusion_matrix(y_true=expected, y_pred=predicted)
confusion2

array([[92,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0],
       [16,  4,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 3,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 2,  0,  0,  1,  1,  0,  0,  0,  1,  0,  0,  0],
       [11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 2,  0,  0,  0,  0,  0,  0,  4,  0,  0,  0,  0],
       [10,  1,  1,  0,  0,  0,  0,  0,  7,  0,  0,  0],
       [ 3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0],
       [ 7,  0,  0,  0,  0,  0,  0,  0,  1,  0,  1,  0]])